# Shift invariance in the DWT domain

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mimg
%matplotlib inline
import pywt

### Create three images
With a circle of diameter 10 that moves from coordinate (30, 20), (30, 21), and (30, 22). (row, column)

In [ ]:
%%bash
/home/vruiz/MRVC/tools/moving_circle.sh -w 64 -h 64 -x 20 -y 30 -d 10 -f 3
mv /tmp/moving_circle_000.png .
mv /tmp/moving_circle_001.png .
mv /tmp/moving_circle_002.png .

### Displaying stuff

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_img(prefix):
    img = load_img(prefix)
    img = normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(img)
    
def load_frame(prefix):
    fn = f"{prefix}.png"
    print(fn)
    frame = cv2.imread(fn, cv2.IMREAD_UNCHANGED) # [rows, columns, components]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.array(frame)
    frame = frame.astype(np.float32) - 32768.0
    return frame

def write_frame(frame, prefix):
    frame = frame.astype(np.float32)
    frame += 32768.0
    frame = frame.astype(np.uint16)
    cv2.imwrite(f"{prefix}.png", frame)

def show_seq(prefix="/tmp/", frames=5):
    for i in range(frames):
        show_img(f"{prefix}{i:03d}")

def show_motion(prefix="/tmp/", frames=5):
    plt.figure(figsize=(10,10))
    prev = load_frame(f"{prefix}000")
    imgplot = plt.imshow(normalize(prev))
    for i in range(1, frames, 1):
        img = load_frame(f"{prefix}{i:03d}")
        diff = img - prev
        prev = img
        diff = normalize(diff)
        plt.figure(figsize=(10,10))
        plt.title(f"{prefix}{i:03d}", fontsize=20)
        plt.imshow(diff)

### Show the sequence

In [ ]:
prefix = "moving_circle_"
show_motion(prefix, frames=3)

### Color transform stuff

In [ ]:
def RGB_to_YCoCg(RGB_frame):
    R, G, B = RGB_frame[:,:,0], RGB_frame[:,:,1], RGB_frame[:,:,2]
    YCoCg_frame = np.empty_like(RGB_frame)
    YCoCg_frame[:,:,0] =  R/4 + G/2 + B/4 
    YCoCg_frame[:,:,1] =  R/2       - B/2
    YCoCg_frame[:,:,2] = -R/4 + G/2 - B/4
    return YCoCg_frame

### Load the images

In [ ]:
frame_0 = load_frame(prefix + "000")
frame_1 = load_frame(prefix + "001")
frame_2 = load_frame(prefix + "002")
write_frame(frame_1-frame_0, "difference_0")
write_frame(frame_2-frame_0, "difference_1")

### Work only with luma

In [ ]:
frame_0_Y = RGB_to_YCoCg(frame_0)[:,:,0]
frame_1_Y = RGB_to_YCoCg(frame_1)[:,:,0]
frame_2_Y = RGB_to_YCoCg(frame_2)[:,:,0]

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 0. Circle at (30, 20)", fontsize=20)
plt.imshow(frame_0_Y, cmap='gray')
plt.savefig("frame_0_Y.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 1. Circle at (30, 21)", fontsize=20)
plt.imshow(frame_1_Y, cmap='gray')
plt.savefig("frame_1_Y.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 2. Circle at (30, 22)", fontsize=20)
plt.imshow(frame_2_Y, cmap='gray')
plt.savefig("frame_2_Y.png")
plt.show()

### Motion estimation (ME) in the pixel domain
Notice that this ME would be used if the original frames where available. This is not the case in MRCV!

In [ ]:
# Plot the first part (from pixel 0 to 39) of the horizontal line (row 35) of the luminance.
plt.title("ME between frames 0 and 1")
plt.plot(frame_0_Y[35, 0:40], label="reference")
plt.plot(np.roll(frame_0_Y[35, 0:40], 1), label="prediction")
#plt.show()
plt.plot(frame_1_Y[35, 0:40], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("movement_0.svg")
#plt.savefig("movement_0.pdf")
plt.show()

In [ ]:
plt.title("ME between frames 0 and 1")
plt.plot(frame_0_Y[35, 0:40], label="reference")
plt.plot(np.roll(frame_0_Y[35, 0:40], 2), label="prediction")
#plt.show()
plt.plot(frame_2_Y[35, 0:40], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("movement_1.svg")
#plt.savefig("movement_1.pdf")
plt.show()

### Haar 2D-DWT

In [ ]:
f0_haar_LL, (f0_haar_LH, f0_haar_HL, f0_haar_HH) = pywt.dwt2(frame_0_Y, 'haar')
f1_haar_LL, (f1_haar_LH, f1_haar_HL, f1_haar_HH) = pywt.dwt2(frame_1_Y, 'haar')
f2_haar_LL, (f2_haar_LH, f2_haar_HL, f2_haar_HH) = pywt.dwt2(frame_2_Y, 'haar')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 0. Haar. LL", fontsize=20)
plt.imshow(f0_haar_LL, cmap='gray')
plt.savefig('f0_haar_LL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 1. Haar. LL", fontsize=20)
plt.imshow(f1_haar_LL, cmap='gray')
plt.savefig('f1_haar_LL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 2. Haar. LL", fontsize=20)
plt.imshow(f2_haar_LL, cmap='gray')
plt.savefig('f2_haar_LL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 0. Haar. LH", fontsize=20)
plt.imshow(f0_haar_LH, cmap='gray')
plt.savefig('f0_haar_LH.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 1. Haar. LH", fontsize=20)
plt.imshow(f1_haar_LH, cmap='gray')
plt.savefig('f1_haar_LH.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 2. Haar. LH", fontsize=20)
plt.imshow(f2_haar_LH, cmap='gray')
plt.savefig('f2_haar_LH.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 0. Haar. HL", fontsize=20)
plt.imshow(f0_haar_HL, cmap='gray')
plt.savefig('f0_haar_HL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 1. Haar. HL", fontsize=20)
plt.imshow(f1_haar_HL, cmap='gray')
plt.savefig('f1_haar_HL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 2. Haar. HL", fontsize=20)
plt.imshow(f2_haar_HL, cmap='gray')
plt.savefig('f2_haar_HL.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 0. Haar. HH", fontsize=20)
plt.imshow(f0_haar_HH, cmap='gray')
plt.savefig('f0_haar_HH.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 1. Haar. HH", fontsize=20)
plt.imshow(f1_haar_HH, cmap='gray')
plt.savefig('f1_haar_HH.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Frame 2. Haar. HH", fontsize=20)
plt.imshow(f2_haar_HH, cmap='gray')
plt.savefig('f2_haar_HH.png')
plt.show()

Notice that the coefficients of the frames 0 and 2 are identical, but this is not true for the frame 1.

In [ ]:
plt.title("Frames 0 and 1. Haar. LL")
plt.plot(np.roll(f0_haar_LL[18, 0:20], 0), label="prediction")
plt.plot(f1_haar_LL[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_1_haar_LL.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 2. Haar. LL")
plt.plot(np.roll(f0_haar_LL[18, 0:20], 1), label="prediction")
plt.plot(f2_haar_LL[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_2_haar_LL.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 1. Haar. LH")
plt.plot(np.roll(f0_haar_LH[18, 0:20], 0), label="prediction")
plt.plot(f1_haar_LH[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_1_haar_LH.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 2. Haar. LH")
plt.plot(np.roll(f0_haar_LH[18, 0:20], 1), label="prediction")
plt.plot(f2_haar_LH[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_2_haar_LH.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 1. Haar. HL")
plt.plot(np.roll(f0_haar_HL[18, 0:20], 0), label="prediction")
plt.plot(f1_haar_HL[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_1_haar_HL.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 2. Haar. HL")
plt.plot(np.roll(f0_haar_HL[18, 0:20], 1), label="prediction")
plt.plot(f2_haar_HL[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_2_haar_HL.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 1. Haar. HH")
plt.plot(np.roll(f0_haar_HH[18, 0:20], 0), label="prediction")
plt.plot(f1_haar_HH[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_1_haar_HH.svg")
plt.show()

In [ ]:
plt.title("Frames 0 and 2. Haar. HH")
plt.plot(np.roll(f0_haar_HH[18, 0:20], 1), label="prediction")
plt.plot(f2_haar_HH[18, 0:20], linestyle=':', linewidth=4, label="predicted")
plt.legend()
plt.savefig("f0_2_haar_HH.svg")
plt.show()

### Daubechies 5 2D-DWT 
sin acabar ...

In [ ]:
decomposition_0_db5 = pywt.dwt2(frame_0_Y, 'db5')
decomposition_1_db5 = pywt.dwt2(frame_1_Y, 'db5')

In [ ]:
plt.title("Coefs in LL (Daubechies 5)")
plt.plot(decomposition_0_db5[0][18, 0:20])
plt.plot(decomposition_1_db5[0][18, 0:20], linestyle=':')
plt.savefig("db5_LL.svg")
plt.savefig("db5_LL.pdf")
plt.show()

In [ ]:
decomposition_0_bior35 = pywt.dwt2(frame_0_Y, 'bior3.5')
decomposition_1_bior35 = pywt.dwt2(frame_1_Y, 'bior3.5')

In [ ]:
plt.title("Coefs in LL (Biorthogonal 3.5)")
plt.plot(decomposition_0_bior35[0][18, 0:20])
plt.plot(decomposition_1_bior35[0][18, 0:20], linestyle=':')
plt.savefig("bior35_LL.svg")
plt.savefig("bior35_LL.pdf")
plt.show()

In [ ]:
plt.title("Coefs in LH (Haar)")
plt.plot(decomposition_0_haar[1][0][18, 0:20])
plt.plot(decomposition_1_haar[1][0][18, 0:20], linestyle=':')
plt.savefig("haar_LH.svg")
plt.savefig("haar_LH.pdf")
plt.show()

In [ ]:
plt.title("Coefs in LH (Daubechines 5)")
plt.plot(decomposition_0_db5[1][0][18, 0:20])
plt.plot(decomposition_1_db5[1][0][18, 0:20], linestyle=':')
plt.savefig("db5_LH.svg")
plt.savefig("db5_LH.pdf")
plt.show()

In [ ]:
plt.title("Coefs in LH (Biorthogonal 3.5)")
plt.plot(decomposition_0_bior35[1][0][18, 0:20])
plt.plot(decomposition_1_bior35[1][0][18, 0:20], linestyle=':')
plt.savefig("bior35_LH.svg")
plt.savefig("bior35_LH.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HL (Haar)")
plt.plot(decomposition_0_haar[1][1][18, 0:20])
plt.plot(decomposition_1_haar[1][1][18, 0:20], linestyle=':')
plt.savefig("haar_HL.svg")
plt.savefig("haar_HL.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HL (Daubechines 5)")
plt.plot(decomposition_0_db5[1][1][18, 0:20])
plt.plot(decomposition_1_db5[1][1][18, 0:20], linestyle=':')
plt.savefig("db5_HL.svg")
plt.savefig("db5_HL.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HL (Biorthogonal 3.5)")
plt.plot(decomposition_0_bior35[1][1][18, 0:20])
plt.plot(decomposition_1_bior35[1][1][18, 0:20], linestyle=':')
plt.savefig("bior35_HL.svg")
plt.savefig("bior35_HL.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HH (Haar)")
plt.plot(decomposition_0_haar[1][2][18, 0:20])
plt.plot(decomposition_1_haar[1][2][18, 0:20], linestyle=':')
plt.savefig("haar_HH.svg")
plt.savefig("haar_HH.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HH (Daubechines 5)")
plt.plot(decomposition_0_db5[1][2][18, 0:20])
plt.plot(decomposition_1_db5[1][2][18, 0:20], linestyle=':')
plt.savefig("db5_HH.svg")
plt.savefig("db5_HH.pdf")
plt.show()

In [ ]:
plt.title("Coefs in HH (Biorthogonal 3.5)")
plt.plot(decomposition_0_bior35[1][2][18, 0:20])
plt.plot(decomposition_1_bior35[1][2][18, 0:20], linestyle=':')
plt.savefig("bior35_HH.svg")
plt.savefig("bior35_HH.pdf")
plt.show()